# 결측치 다루기

데이터 출처: [Ramen Rating(Kaggle)](https://www.kaggle.com/residentmario/ramen-ratings), [Everything you can do with a time series(kaggle)](https://www.kaggle.com/kernels/scriptcontent/4882227/notebook)

## Preparation

In [ ]:
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
ramen = pd.read_csv("./data/ramen-review/ramen-ratings.csv")
ramen["Stars"] = pd.to_numeric(ramen["Stars"], errors="coarse")
google = pd.read_csv('./data/stock-time-series-20050101-to-20171231/GOOGL_2006-01-01_to_2018-01-01.csv')

In [ ]:
ramen.head()

In [ ]:
ramen.dtypes

In [ ]:
google.head()

In [ ]:
google.dtypes

## 결측치

pandas는 numpy의 nan을 NaN 값으로 사용한다.

In [ ]:
ramen["Top Ten"].head()

nan인지 확인하기 위해서는 `isna()` 또는 `notna()`를 사용한다.

In [ ]:
ramen["Top Ten"].isna().head()

In [ ]:
ramen["Top Ten"].notna().head()

> (주의) `ramen["Top Ten"] == np.nan`은 패키지 내부의 문제 때문에 정상적으로 작동하지 않는다.

In [ ]:
# True가 되어야 정상
(ramen["Top Ten"] == np.nan).head()

## Missing Value Propagation

결측치와 어떤 연산을 하더라도 결과는 항상 `nan`이 된다. 이런 특성 때문에 결측치가 포함된 상태로 계산을 하면 결측치가 전파된다.

In [ ]:
np.nan + 3

In [ ]:
np.nan * 2

전파되는 예시로 구글 시가의 이동평균을 구할 때 결측치가 있으면 어떤 결과가 나타나는지 본다.

In [ ]:
google["Open"].head()

In [ ]:
# 결측치 추가하기
tmp = google["Open"][4]
google.loc[4,"Open"] = np.nan
google["Open"].head()

결측치를 추가하고 3일 이동평균을 구해본다.

In [ ]:
# 결측치 추가하기
google.set_index("Date")["Open"].rolling(window=3).mean().head(15)

`Open` 컬럼의 4번째 값 결측치때문에 이동평균에 3개의 NaN 값이 생긴다.

## 결측치 처리

결측치 처리는 보통 둘 중 하나로 처리된다.

1. 결측치가 있는 row나 column을 버린다.
2. 결측치를 채운다.(interpolate 또는 imputation)

### 결측치 버리기

`dropna()`를 사용한다.

In [ ]:
google.head(10)

In [ ]:
# NaN이 들어있는 row 버리기 (4번째 row 사라짐)
google.dropna().head(10)
# 또는 google.dropna(axis=0).head(10)

In [ ]:
# NaN이 들어있는 column 버리기 
# (결측치가 들어있는 Open 컬럼 없어짐)
google.dropna(axis=1).head(10)

### 결측치 채우기

`interpolate()`를 사용한다. 여러 방식의 interpolate을 제공하는데 `method=`형태로 지정할 수 있다. method는 다음과 같은 방식을 사용할 수 있다.

* linear
* time
* index
* values
* nearest
* zero
* slinear
* quadratic
* cubic
* barycentric

아래 method는 scipy가 같이 설치되어 있을 때만 사용 가능하다.
* krogh
* polynomial
* spline
* piecewise_polynomial
* pchip

자세한 내용은 [DataFrame.interpolate reference](https://pandas.pydata.org/pandas-docs/version/0.25/generated/pandas.DataFrame.interpolate.html)를 참고하면 된다.

In [ ]:
# 예시 데이터
sin = pd.DataFrame({"A":np.sin(np.linspace(-np.pi, np.pi, 25))})
sin[5:12] = np.nan
sin.plot(style='.')
plt.show()

In [ ]:
# 여러 메소드 비교하기
NCOL = 2
NROW = 5
fig, axes = plt.subplots(NROW, NCOL)
fig.set_size_inches(10, 20)
methods = [
    "linear", "nearest", "slinear", "cubic", "quadratic", "spline", "spline",
    "spline", "pchip", "akima"
]
args = [{}, {}, {}, {}, {}, {"order": 1}, {"order": 2}, {"order": 3}, {}, {}]
for i in range(NROW):
    for j in range(NCOL):
        idx = i * NCOL + j
        sin.plot(style='.', ax=axes[i][j], legend=False)
        sin["A"] \
            .interpolate(methods[idx], **args[idx]) \
            .iloc[5:12] \
            .plot(style='r.', ax=axes[i][j], legend=False)
        axes[i][j].set_title(methods[idx] + str(args[idx]))
plt.show()

## Casting rules

|Data|Type|
|:-----|:-----|
|1, 2, 3, 4, 5, …|integer|
|1,2,NaN,4,5|float|
|Ture, False, True, ...|boolean|
|True, NaN, False|Object(String 타입)|
|1.2, 1.3, NaN, 1.9|float|

`pd.read_csv()`와 같이 데이터를 로드하는 함수를 실행할 때 pandas가 자체적으로 타입을 추론한다. 단, 로드하는 column에 결측값이 있으면 다음과 같은 Casting rule에 따라 타입이 결정된다.

In [ ]:
ramen.loc[100,"Review #"] = np.nan
ramen.to_csv("./data/ramen-review/ramen-ratings2.csv",index=False)

In [ ]:
ramen = pd.read_csv("./data/ramen-review/ramen-ratings2.csv")

In [ ]:
ramen.head()

In [ ]:
ramen.dtypes

`Review #` 컬럼은 리뷰의 개수를 의미하기 때문에 int 타입으로 되어야 하는데 float64 타입으로 인식되었으므로 nan이 포함되어 있을 수 있음을 의심할 수 있다.

In [ ]:
ramen["Review #"][ramen["Review #"].isna()]

casting rule을 알고 있으면 결측값의 존재한다는 사실을 빨리 알아차릴 수 있다. (데이터를 읽은 후에 결측값 체크를 하는 것이 통상적인 절차라 사실 몰라도 상관 없다.)

In [ ]:
ramen.isna().sum()

위의 코드로 모든 컬럼의 결측치를 쉽게 파악할 수 있다. `sum()`을 사용한 것은 `True`는 숫자로 표현하면 1, `False`는 0이라는 성질을 이용하기 위해서다. 